# Automated Substation Creation and Feeder Insertion in Native CIM17

CIM-Builder is a new python library developed under the Grid Atlas and MAPLE LEAF projects for creating CIM models "from scratch" with no pre-existing model files. This is a significantly different capability than all other tooling, which requires a source file from which to start, such as OpenDSS, PSSE, or GIS data. The library currently inlcudes three main functionalities:

1) automatic creation of new node-breaker substations in CIM based on interactive API calls

2) automatic insertion of existing distribution feeders in CIM into new node-breaker substations

3) automatic insertion of new aggregate feeder data in existing CIM transmission models (not demonstrated in this report). //

The library contains a set of classes for each type of common substation, which each contain two methods. The first is `add_feeder`, which automatically adds the specified distribution feeder to the substation and creates the chain of switching equipment between the sourcebus of the feeder and the correct breaker or airgap switch in the substation. The second is `add_branch`, which enables automated addition of transmission branches, transformers, and shunt equipment to the substation bus.

The classes supported within the first version of CIM-Builder are

* `SingleBusSubstation`
* `MainAndTransferSubstation`
* `RingBusSubstation`
* `DoubleBusSingleBreakerSubstation`
* `BreakerAndHalfSubstation`

When instantiated, these classes create a new CIMantic Graphs `DistributedArea` or `NodeBreakerModel` graph model with all CIM objects associated with the default bus configuration for that substation. Distribution feeders can then be added to the substation by instantiating each feeder as a new CIMantic Graphs `FeederModel`. The python library then builds the set of CIM associations to map the feeder to the substation and create all CIM objects for the breaker, airgap switches, and junctions.



----

### Import CIMantic Graphs Library and CIM profile

The CIMantic Graphs library can be installed using `pip install cim-graph` and imported into any script:

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
import cimgraph.utils as utils

CIMantic Graphs can support multiple CIM profiles, so it is necessary to specify which one to use. This demonstration will use the extensions from CIMHub feature/SETO, which is derived from CIM17V38.

In [2]:
# Specify CIM Profile
import importlib
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

To create a new model "from scratch", no filename or databased is specified. This connection object contains all information regarding the CIM version, serilaization format, etc.

In [3]:
# Create an empty connection for new file
params = ConnectionParameters(filename=None,
                              cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

For this demonstration, the IEEE 13 and IEEE 13 assets feeders will be used to populate each substation. It is possible to load these files from a database, or open them directly from the XML using CIMantic Graphs. Since end-users of the Grid Atlas may not have a database, the feeders will be loaded from the standalone files:

In [4]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml',
                               cim_profile=cim_profile, iec61970_301=8)
ieee13_connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=ieee13_connection, container=ieee13_feeder,
                              distributed=False)


In [5]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', 
                              cim_profile=cim_profile, iec61970_301=8)
assets13_connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=assets13_connection, container=assets13_feeder,
                                distributed=False)


For this demonstration, the Blazegraph database used by GridAPPS-D will be used for roundtrip testing. During these tests, each of the substation models developed for Grid Atlas will be added to the database. Connectivity and total load within the test feeders will be queried through a simple test.

To connect to the database, we will establish two connections. The first will be used for database bulk upload using the CIM-Loader library previously developed for Grid Atlas. The second will be used for automated database queries using CIMantic Graphs.

In [6]:
# Connection to Blazegraph Database for CIM-Loader
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
                               cim_profile=cim_profile, iec61970_301=8)
bg_loader = BlazegraphLoader(params)

In [7]:
# Connection to Blazegraph Database for CIMantic Graphs
from cimgraph.databases import BlazegraphConnection as BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
                               cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)

### Create New Single Bus Substation

To create a new Single Bus substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [8]:
from cimbuilder.substation_builder import SingleBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [9]:
SubBuilder = SingleBusSubstation(connection=connection,
                                 name="single_bus_sub", 
                                 base_voltage = 115000)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches.The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [10]:
SubBuilder.new_feeder(series_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(series_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [11]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "1672f75d-4487-4270-8bbd-090aa2979db6": {
        "mRID": "1672f75d-4487-4270-8bbd-090aa2979db6",
        "name": "single_bus_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9E

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [12]:
utils.write_xml(SubBuilder.network, '../test_output/single_bus.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [13]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=306ms, elapsed=234ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=385ms, commitTime=1709690418951, mutationCount=6789</p\n></html\n>'

In [14]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/single_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 63
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="478"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 63
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="140"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="173" milliseconds="47"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [15]:
network = NodeBreakerModel(container=substation, connection=blazegraph)

In [16]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "1672f75d-4487-4270-8bbd-090aa2979db6": {
        "mRID": "1672f75d-4487-4270-8bbd-090aa2979db6",
        "name": "single_bus_sub",
        "ConnectivityNodes": "['8948f0c1-8387-47fd-825e-9e1abb78e8c5', 'd27b70f9-80c4-4874-b32e-b6bee2f9c0c5', '4cabedb5-1154-4025-9ca5-2db23a824c10', '601add86-558f-4c2a-a0e3-5c687a55490e', '675230a7-48b0-44ec-bdcb-1e41ffe7be0b']",
        "Equipments": "['87d04109-9826-4443-9e33-dc283abb4216', '87db4e0a-50ab-41a2-90c9-594a595d3745', '8bfd2c69-f830-4059-a21c-86ae10fd76d6', 'bc177e50-f2f3-446e-8073-2f331f4c579c', 'bf9669e7-862c-4452-9531-849f4ce485fa', 'f9645444-5ab9-40c2-9eaf-bc8965789925', '609c0f5d-bf47-4673-acf8-dbaee4f891c0']",
        "NormalEnergizedFeeder": "['49AD8E07-3BF9-A4E2-CB8F-C3722F837B62', '5B816B93-7A5F-B64C-8460-47C17D6E4B0F']"
    }
}


In [17]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [18]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [19]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/single_bus_and_feeders.xml')

### Create New Sectionalized-Bus Substation

To create a new Sectionalized-Bus substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [20]:
from cimbuilder.substation_builder import SectionalizedBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of bus sections is specified by `total_sections` with a default value of two bus sections.

In [21]:
SubBuilder = SectionalizedBusSubstation(connection=connection,
                                        name="sectionalized_sub", 
                                        base_voltage = 115000,
                                        total_sections = 2)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `section_number` is used to identify the bus section to which the feeder should be connected. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [22]:
SubBuilder.new_feeder(section_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(section_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [23]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "28cfce9c-3568-465a-913b-029437bc7147": {
        "mRID": "28cfce9c-3568-465a-913b-029437bc7147",
        "name": "sectionalized_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [24]:
utils.write_xml(SubBuilder.network, '../test_output/sectionalized.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [25]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=39ms, elapsed=38ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=48ms, commitTime=1709690424642, mutationCount=6461</p\n></html\n>'

In [26]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/sectionalized.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="56"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="75"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="252" milliseconds="45"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [27]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [28]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "28cfce9c-3568-465a-913b-029437bc7147": {
        "mRID": "28cfce9c-3568-465a-913b-029437bc7147",
        "name": "sectionalized_sub",
        "ConnectivityNodes": "['d368f54a-29c3-4304-acf0-6cf92cff3538', 'ef68d6a2-723f-4f4f-b20d-1f883cb7f900', '1ecb430c-2383-4ba8-a0f2-9fcb7edb128f', '29479bd1-99e6-4bb7-9e4d-1a2414ded66c', '40131e52-8338-4a53-932e-7975fd544ba3', '4e7cf8a1-f53b-41ce-9ec4-2872a0e715cb', '664ef03a-ddd7-408b-92f4-ee1cc91d7f28', '6c0a1c47-e242-45ce-a7c6-21795dcd5536']",
        "Equipments": "['c1258bdf-c6a1-4ef2-bb93-5091c93346ba', 'e22531ed-ef7e-4282-a560-ceeeb208afb2', 'eb9d6435-caa3-4e04-b48b-541ffd6fa4cb', '242877d8-bdd3-4731-89d5-8b1c80568b19', '2779aa15-e896-413a-85ac-9b204a77e8b4', '42ede277-14b0-48eb-b228-8bb1f4628e5b', '53859b2e-16ea-494a-a0ba-16edb0796a25', '6b625e97-ef49-42bc-90e4-4b9bd5ed0718', '7013c0dd-19aa-4562-9094-f243e2c56fef', '7541450d-b429-462c-b1da-0a235718d1d2', '791065e5-6c37-4d0d-882f-20c45a0b330b']",
        "NormalEnergizedFeeder": "['49AD

In [29]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [30]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [31]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/sectionalized_and_feeders.xml')

### Create New Main-and-Transfer Substation

To create a new Main and Transfer substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [32]:
from cimbuilder.substation_builder import MainAndTransferSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [33]:
SubBuilder = MainAndTransferSubstation(connection=connection,
                                       name="main_transfer_sub", 
                                       base_voltage = 115000)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [34]:
SubBuilder.new_feeder(series_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(series_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [35]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "46e7633d-6b2e-4043-9634-a0b8a6102992": {
        "mRID": "46e7633d-6b2e-4043-9634-a0b8a6102992",
        "name": "main_transfer_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [36]:
utils.write_xml(SubBuilder.network, '../test_output/main_and_transfer.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [37]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=23ms, elapsed=23ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=28ms, commitTime=1709690429116, mutationCount=6540</p\n></html\n>'

In [38]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/main_and_transfer.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="36"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="32"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="290" milliseconds="38"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [39]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [40]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "46e7633d-6b2e-4043-9634-a0b8a6102992": {
        "mRID": "46e7633d-6b2e-4043-9634-a0b8a6102992",
        "name": "main_transfer_sub",
        "ConnectivityNodes": "['8551d222-753b-4b95-b86f-3fe3356c27e3', '9a971917-ad2f-4195-afe2-30d6ee991248', 'a6321443-b82d-4b06-aa85-760fb1394f91', 'b60bf66b-eee8-4f32-bea5-4b97b3d27540', 'd4020cbc-9a8d-49d9-ae0f-b6db9f38125d', 'd6745a8d-aeb3-4229-a101-a882736eba1f', 'e94c2c34-7dfb-4786-96ba-c3859f3fecc5', '152b78c2-fce9-49e7-8c17-b746be0421d1']",
        "Equipments": "['a5fcad02-595f-47f9-a72c-89113297cf74', 'a949e364-41bd-45eb-bf26-d0513062f89f', 'c2055188-f426-40c6-9ae4-bb8ceae1d8e4', 'c3590b3f-7b22-49b0-a144-d24995129475', 'c3eac626-1714-4d7d-9e08-58bc1dcd50a5', 'e17d2b73-f026-482d-9395-fff7c2bdf813', '097bbe19-f511-4a48-a990-084e4a7cabe9', '0cc614eb-aa1b-4bb1-8cb2-34e5a3ae0ac4', '16e22982-8ed8-4c41-92c2-eb4b496f62d9', '16eccc7d-c213-4375-9031-6f730fc48b21', '3b28b566-1b64-409b-be74-51d69c4eff95', '3d0fb2b2-a537-4fdb-bba0-19683e2a95ce', '4

In [41]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [42]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [43]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/main_transfer_and_feeders.xml')

### Create Double Bus Single Breaker Substation

To create a new Main and Transfer substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [44]:
from cimbuilder.substation_builder import DoubleBusSingleBreakerSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [45]:
SubBuilder = DoubleBusSingleBreakerSubstation(connection=connection,
                                              name="double_bus_sub", 
                                              base_voltage = 115000)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [46]:
SubBuilder.new_feeder(series_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(series_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [47]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "1cde3a3a-e19f-4129-925c-2c7a084f3d09": {
        "mRID": "1cde3a3a-e19f-4129-925c-2c7a084f3d09",
        "name": "double_bus_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9E

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [48]:
utils.write_xml(SubBuilder.network, '../test_output/double_bus.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [49]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=18ms, elapsed=18ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=22ms, commitTime=1709690433356, mutationCount=6578</p\n></html\n>'

In [50]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/double_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="27"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="36"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="244" milliseconds="29"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [51]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [52]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "1cde3a3a-e19f-4129-925c-2c7a084f3d09": {
        "mRID": "1cde3a3a-e19f-4129-925c-2c7a084f3d09",
        "name": "double_bus_sub",
        "ConnectivityNodes": "['9a0b5746-06f4-4a10-9548-0d57cf308b82', 'c39e0d7b-2569-4232-a399-23e7f596f8c7', '078d300d-7287-49d4-995c-1e628d0e34d4', '4ebf1123-40af-441e-9bd6-000251777096', '534f835a-26d3-47a7-946d-84be0cf17196', '7b0ac837-74fe-4559-97a4-2e839fae83b7']",
        "Equipments": "['805f98fd-dba0-4c9a-9833-26aa144bee36', '9f89232b-6913-4cbc-8c8e-fda9394e0cc5', 'b7531bbb-e49c-4340-9565-78b0462a4c1a', 'cbf0033e-1243-4586-adb1-6c04f2e4e9e6', 'd30c3697-0409-424b-9ab0-894c7480b030', 'd394c5fd-285a-4a44-96d9-5f8607338c8a', 'f627e2c8-beb5-443a-a3ef-ad21bb2c2e3c', '0e40636e-d34c-44b5-9580-ad1dea3f28f3', '13073494-8321-4dd0-b509-3661a9259ca0', '34e2df6a-57e4-4a76-9be4-60346500371c', '4ec136f7-0bda-4a8b-8f7b-573176cd6387']",
        "NormalEnergizedFeeder": "['49AD8E07-3BF9-A4E2-CB8F-C3722F837B62', '5B816B93-7A5F-B64C-8460-47C17D6E4B0F']"
    }
}

In [53]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [54]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [55]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/double_bus_and_feeders.xml')

### Create Ring Bus Substation

To create a new Main and Transfer substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [56]:
from cimbuilder.substation_builder import RingBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of bus sections within the ring is denoted by `total_sections`.

In [57]:
SubBuilder = RingBusSubstation(connection=connection,
                               name = "ring_bus_sub", 
                               base_voltage = 115000,
                               total_sections = 4)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [58]:
SubBuilder.new_feeder(bus_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(bus_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [59]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "a1b18b31-8d47-44ff-8ed0-3f6f716765f5": {
        "mRID": "a1b18b31-8d47-44ff-8ed0-3f6f716765f5",
        "name": "ring_bus_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED0

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [60]:
utils.write_xml(SubBuilder.network, '../test_output/ring_bus.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [61]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=31ms, elapsed=31ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=35ms, commitTime=1709690437549, mutationCount=6532</p\n></html\n>'

In [62]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/ring_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="20"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="21"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="383" milliseconds="29"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [63]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [64]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "a1b18b31-8d47-44ff-8ed0-3f6f716765f5": {
        "mRID": "a1b18b31-8d47-44ff-8ed0-3f6f716765f5",
        "name": "ring_bus_sub",
        "ConnectivityNodes": "['c08ba81a-5266-439e-87a2-1649a9f82c33', 'd2a97669-eda2-4948-83eb-e714906d5ff4', 'e48be6ab-e169-4e91-b1fc-d447617feb8f', 'effc4d06-049f-45e5-a6d8-98c63b36d2be', '1c6ac114-85fa-4b6d-9a9e-7b772ed1fc0c', '23387641-9b3c-49cd-9af5-2ed2fb6da11e', '3a0b4757-d8db-4f7b-89b3-7c98ad65c971', '5341b993-9ff6-4d11-9f7b-5cbd95b20b60', '6e7caef2-f2c5-4099-9285-cecaaa5d1ba7', '6f071a9a-bb43-45f4-ad92-f561f16d1225', '79052f05-3081-449e-af38-d3a18bfffe45', '7a571974-7a72-4079-bc0f-a9b2956e6eef']",
        "Equipments": "['9776c7cc-eac7-429a-b17e-03f3756af413', 'a7444c60-adfd-498e-8c9a-322a1c481916', 'b5185dc7-f0fa-4bb1-8dd3-e16a2fdf4d6f', 'b7560811-ec1d-4fc4-9ee9-212871f6cad8', 'c4e83a27-8d34-413c-9726-0d57cc8a999b', 'dae6c7e2-1e3a-4178-81d0-562a07fd3da0', 'dda6676a-2571-479e-be0a-fac6dbe19b28', '09abb7e7-af22-414b-9145-68bdc0303865', '1992ec

In [65]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [66]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [67]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/ring_bus_and_feeders.xml')

### Create Breaker and a Half Substation

To create a new Breaker-and-a-Half substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [68]:
from cimbuilder.substation_builder import BreakerAndHalfSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of breaker sets in the substation is denoted by `total_bus_ties`.

In [69]:
SubBuilder = BreakerAndHalfSubstation(connection=connection,
                               name = "breaker_half_sub", 
                               base_voltage = 115000,
                               total_bus_ties=2)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [70]:
SubBuilder.new_feeder(branch_number=1, tie_number=1,
                      feeder=ieee13_feeder,
                      feeder_network=ieee13_network)
SubBuilder.new_feeder(branch_number = 2, tie_number=1,
                      feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [71]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "2336a778-291f-4016-8fe9-a7bb0dae265c": {
        "mRID": "2336a778-291f-4016-8fe9-a7bb0dae265c",
        "name": "breaker_half_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [72]:
utils.write_xml(SubBuilder.network, '../test_output/breaker_half.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [73]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=18ms, elapsed=18ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=22ms, commitTime=1709690441781, mutationCount=6671</p\n></html\n>'

In [74]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/breaker_half.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="22"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="40"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="501" milliseconds="49"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [75]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [76]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "2336a778-291f-4016-8fe9-a7bb0dae265c": {
        "mRID": "2336a778-291f-4016-8fe9-a7bb0dae265c",
        "name": "breaker_half_sub",
        "ConnectivityNodes": "['841593ae-5059-4e58-ac4d-bf33aee451e9', '923d707e-7897-4201-8e84-5d448dac19cb', 'ad273997-8310-4f4a-a9ca-b150eea3cb33', 'bba8971c-6a90-4774-ab36-b075dbca479a', 'bc0e4c3a-90f7-4c03-9873-6634d3fd0a9f', 'c6d4ffef-a973-4004-a578-f29b01db89fe', 'c9a16f38-f37e-4735-8f7b-3bb29d12af29', 'f473eb2b-b52c-466d-a0ec-44b4271a7b77', '08577179-cd1e-46d9-bbb0-fd7ec8df8888', '14179489-0232-4f06-82f2-eedb9cf3c2f4', '17e64884-953f-43a1-8e3c-b670703b9776', '28849487-03d8-4109-a1cb-7e1ea4a4b596', '30786787-458a-4bf9-97ca-ec289418579d', '3e38ec94-6446-45e9-a814-068402296b66', '494c9eaf-00fd-47fa-b5cb-04ae942aecfc', '599e2e0e-459e-4002-bb48-21053a2f40c7', '787bc5ef-7a05-48af-921f-924fe98b4c7a', '7e81b43f-83d2-45aa-bdfd-18bac5d683a1']",
        "Equipments": "['86404f3c-a294-4607-822b-2ea26a6ee42a', '8d6b5056-00fe-424d-a35a-12ced05f8760', 'b0

In [77]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [78]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [79]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/breaker_half_and_feeders.xml')
